In [ ]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src import *
# from src.data import *
from src.space import *
from src.mcmc import *

# https://dvr.capitol.texas.gov/
level    = 'cntyvtd'
# level    = 'tabblock'
proposal = 'plans2168'  # for 2010 enacted, use 'planc2100' for US congress, 'plans2100' for TX Senate, or 'planh2100' for TX House
contract = 0
# contract = 'proposal'

# D = Data(refresh_tbl=('proposals'))
M = MCMC(level=level, proposal=proposal, contract=contract,
         max_steps   = 0,
         random_seed = 5,
         defect_cap  = 100,
         pop_deviation_target=10,
         )
# M.run_chain()
# M.post_process()

In [ ]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src.mcmc import *

D = Data(refresh_tbl=('proposals'))

In [2]:
s = 'hi'
s.ljust(10, '0')

'hi00000000'

In [ ]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src.mcmc import *

def get_proposal_stats(proposal, level='tabblock', contract=0):
    print(proposal)
    M = MCMC(level=level, proposal=proposal, contract=contract)
    print(proposal, M.defect, M.pop_deviation)
    return M
    
for proposal in ['planc2193', 'plans2168', 'planh2316']:
    M = get_proposal_stats(proposal, level='bg', contract=0)

# D = Data(refresh_tbl=('proposals'))
# df = read_table(f'{data_bq}.TX_2020_proposals')#.query('complete')
# for i, row in df.iterrows():
#     if row['district_type'] == 'cd' and int(row['proposal'][-4:])<=2100:
#         M = get_proposal_stats(row['proposal'], level='tabblock', contract=0)
#         df.loc[i, 'defect'] = M.defect
#         df.loc[i, 'pop_deviation'] = M.pop_deviation

In [ ]:
try:
    import pandas_bokeh
except:
    os.system('pip install --upgrade pandas-bokeh')
    import pandas_bokeh
pandas_bokeh.output_notebook()

plan_cols = ['geoid', 'plan', 'district']
query = f"select {', '.join(plan_cols)} from {M.output}"
plans = run_query(query).pivot(index='geoid', columns='plan', values='district')
repl = {c:str(c) for c in plans.columns}
plans.rename(columns=repl, inplace=True)

node_cols = ['geoid', 'polygon', 'county', 'total_pop', 'aland', 'density', 'polsby_popper']
query = f"select {', '.join(node_cols)} from {M.nodes}"
nodes = run_query(query).set_index('geoid')
geo = gpd.GeoSeries.from_wkt(nodes['polygon'], crs=crs_census).simplify(0.003).buffer(0) #<-- little white space @ .001 ~5.7 mb, minimal at .0001 ~10mb, with no white space ~37mb

In [ ]:
recolor_seed = 10
rng = np.random.default_rng(recolor_seed)
recolor = dict(zip(M.districts, rng.permutation(M.districts)))
gdf = gpd.GeoDataFrame(nodes
                       .drop(columns='polygon')
                       .join(plans.replace(recolor), how='inner')
                       , geometry=geo).reset_index()

In [ ]:
width = 600
xlim = [-106.2, -94.0]
ylim = [ 25.4 ,  36.6]
aspect = (ylim[1] - ylim[0]) / (xlim[1] - xlim[0])
height = round(aspect * width)

def f(cmap):
    fn = str(M.gpickle)
    fn = f'{fn[:fn.rfind("_")]}_{M.random_seed}_animation_{cmap}.html'
    print(f'cmap={cmap} ... saving animation to {fn}')
    fig = gdf.plot_bokeh(
        colormap = cmap,
        hovertool_string = '@geoid, @county<br>pop=@total_pop<br>density=@density{0.0}<br>land=@aland{0.0}<br>pp=@polsby_popper{0.0}',
        fill_alpha = 0.5,
        line_alpha = 0.05,
        return_html = True,
        show_figure = True,
        slider = list(repl.values()),
        slider_name = "PLAN",
        show_colorbar = False,
        xlim = xlim,
        ylim = ylim,
        figsize = (width, height),
    )
    with open(fn, 'w') as file:
        file.write(fig)
    return fig

f('Paired')

# for cmap in ['Accent', 'Blues', 'Bokeh', 'BrBG', 'BuGn', 'BuPu', 'Category10', 'Category20', 'Category20b', 'Category20c',
#              'Cividis', 'Colorblind', 'Dark2', 'GnBu', 'Greens', 'Greys', 'Inferno', 'Magma', 'OrRd', 'Oranges', 'PRGn',
#              'Paired', 'Pastel1', 'Pastel2', 'PiYG', 'Plasma', 'PuBu', 'PuBuGn', 'PuOr', 'PuRd', 'Purples', 'RdBu', 'RdGy',
#              'RdPu', 'RdYlBu', 'RdYlGn', 'Reds', 'Set1', 'Set2', 'Set3', 'Spectral', 'Turbo', 'Viridis', 'YlGn', 'YlGnBu',
#              'YlOrBr', 'YlOrRd']:

#     f(cmap)

In [ ]:
# P.sort_values('seats', ascending=False).to_csv('/home/jupyter/hi.csv')
# P[P['seats']>0.1].sort_values().to_csv('/home/jupyter/hi.csv')
A = P['seats']
B = A[A>0.1].sort_values(ascending=False)
B.to_frame()
# .to_csv('/home/jupyter/hi.csv')

In [ ]:
P = pd.DataFrame({x:d for x, d in M.adj.nodes(data=True) if x in M.counties}).T[['seats']]
S = P.copy()
for c in range(11):
    S[f'contract_{c}'] = S['seats'] < c /10
S = S.drop(columns='seats')
Q = pd.DataFrame()
Q['not contracted'] = (~S).sum (axis=0)
Q['contracted'] = S.sum (axis=0)
Q['contracted_pct']   = np.round(S.mean(axis=0) * 100).astype(int)

# Q = P.sum(axis=0).to_frame()
# Q['pct']=
Q
# P.mean(axis=0)*100
# P

In [ ]:
county_pop = {x:d for x, d in M.adj.nodes(data='seats') if x in M.counties}
P = list(county_pop.values())
x = np.linspace(0,1,11)
np.less.outer(P, x).shape
# {pop:(county_pop.values()<pop).sum()*100 for pop in 

In [ ]:
[d for x, d in M.adj.nodes(data='seats') if x in M.counties]np.sort([d for x, d in M.adj.nodes(data='seats') if x in M.counties])
np.histogram([d for x, d in M.adj.nodes(data='seats') if x in M.counties], np.linspace(0,1,11))

In [ ]:
from statsmodels.distributions.empirical_distribution import ECDF
# pd.DataFrame({x:d for x, d in M.adj.nodes(data=True) if x in M.counties}).T['seats'].cumsum()
([d for x, d in M.adj.nodes(data='seats') if x in M.counties])

In [ ]:
e.y